In [1]:
# Library ----------------------------------------------------------------
library(dplyr)
library(tidyr)
library(lubridate)

# 1. Load Data, Clustering Results and Scale Vectors ---------------------------

kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_kmeans.rds")

appData <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/01_app_2022.rds")

# pour process_app_data
scale_origins <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_scale_origins.rds")  # Moyennes
scale_ecarts_types  <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/03_pilot1_scale_ecarts_types.rds")  # Écarts-types

In [21]:
# Fonction pour assigner les clusters à toutes les données de l'app
assign_clusters_to_app_data <- function(app_data, scale_origins, scale_ecarts_types, centroids) {
  # 1. Préparer et normaliser les données de l'app
  app_data_processed <- process_app_data(app_data, scale_origins, scale_ecarts_types)
  
  # 2. Exclure vote_intent pour le calcul des distances
  feature_cols <- setdiff(names(app_data_processed), "vote_intent")
  data_features <- app_data_processed[, feature_cols, drop = FALSE]
  centroids_features <- centroids[, feature_cols, drop = FALSE]
  
  # 3. Assigner chaque répondant au cluster le plus proche
  cluster_assignment <- assign_clusters(data_features, centroids_features)
  
  # 4. Ajouter les clusters aux données
  app_data_processed$cluster <- cluster_assignment
  
  return(app_data_processed)
}

# Appliquer la fonction à toutes les données de l'app
app_data_with_clusters <- assign_clusters_to_app_data(app_data, scale_origins, scale_ecarts_types, centroids)

In [22]:
app_data_with_clusters <- assign_clusters_to_app_data(app_data, scale_origins, scale_ecarts_types, centroids)

In [23]:
write_rds(app_data_with_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/qc2022/05_app_2022_clustered.rds")